# TAR: Proyecto Final 

In [ ]:
%pip install gymnasium pandas numpy

## Tratamiento de datos

In [1]:
import numpy as np
import pandas as pd
import os

In [ ]:
ruta_datosMOP = "./Datos/MOP"
ruta_datosADME = "./Datos/ADME"
ruta_datos_procesados = "./Datos_procesados"

## Entorno personalizado para el problema hidro-térmico

In [ ]:
import gymnasium as gym
from gymnasium import spaces

T_MAX = 52
V_MAX = 0
N_HIDRO = 0
V0 = 0

def inicial_hidrologia(): ...

def siguiente_hidrologia(tiempo, hidrologia_actual): ...

def costo_termico(demanda, qt, vt): return 0

def ingreso_exportacion(exportacion): return 0

class HydroThermalEnv(gym.Env):
    def _vertido(self, qt):
        return np.max(self.v - qt + self._aportes() - V_MAX, 0)
    
    def _aportes(self): ...
    
    def _demanda(self): ...
    
    def _gen_eolico(self): ...

    def _gen_solar(self): ...

    def _gen_bio(self): ...

    def _despachar(self, qt): ...

    def __init__(self):
        # Defino espacios de observación y acción
        self.observation_space = spaces.Dict({
            "volumen": spaces.Box(0.0, V_MAX, shape=()),
            "hidrologia": spaces.Discrete(N_HIDRO),
            "tiempo": spaces.Discrete(52)
        })
        # turbinar en [0, V_MAX] continuo
        self.action_space = spaces.Box(0.0, V_MAX, shape=())

    def reset(self, *, seed=None, options=None):
        self.v = V0
        self.t = 0
        self.h = inicial_hidrologia()
        info = {
            "volumen_inicial": self.v,
            "hidrologia_inicial": self.h,
            "tiempo_inicial": self.t
        }
        return self._get_obs(), info
    
    def step(self, action):
        # Validar que la acción esté en el espacio válido
        assert self.action_space.contains(action), f"Acción inválida: {action}. Debe estar en {self.action_space}"
        qt = float(action)
        
        # dinámica: v ← v − q − d + a
        self.v = self.v - qt - self._vertido(qt) + self._aportes()
        self.h = siguiente_hidrologia(self.t, self.h)
        self.t += 1

        # despacho: e_eolo + e_sol + e_bio + e_termico + e_hidro = dem + exp
        exportacion = self._despachar(qt)

        # recompensa = −costo_termico() + ingreso_exportacion()
        reward = -costo_termico(self._demanda(), qt, self.v) + ingreso_exportacion(exportacion)
        
        done = (self.t >= T_MAX)
        info = {
            "turbinado": qt,
            "volumen": self.v,
            "hidrologia": self.h,
            "vertido": self._vertido(qt),
            "exportacion": exportacion,
        }
        return self._get_obs(), reward, done, False, info
    
    def render(self, mode='human'):
        if mode == 'human':
            print(f"Semana {self.t}/52:")
            print(f"  Volumen embalse: {self.v:.2f}/{V_MAX}")
            print(f"  Estado hidrológico: {self.h}")
            print(f"  Porcentaje llenado: {(self.v/V_MAX)*100:.1f}%")
            print("-" * 30)
        elif mode == 'rgb_array':
            # Retornar una imagen como array numpy para grabación
            pass
        elif mode == 'ansi':
            # Retornar string para mostrar en terminal
            return f"T:{self.t} V:{self.v:.1f} H:{self.h}"
        
    def _get_obs(self):
        # Mapeo de variables internas a observación del agente
        obs = {
            "volumen": self.v, 
            "hidrologia": self.h, 
            "tiempo": self.t
        }
        # Validar contra observation_space (opcional, útil para debug)
        assert self.observation_space.contains(obs), f"Observación inválida: {obs}. Debe estar en {self.observation_space}"
        return obs